In [1]:
import pickle
from copy import deepcopy

In [2]:
with open('tic_tac_toe_weights_agent_1.pkl', 'rb') as f:
    weights_agent_1 = pickle.load(f)
with open('tic_tac_toe_weights_agent_2.pkl', 'rb') as f:
    weights_agent_2 = pickle.load(f)

In [3]:
class TicTacToe:
    def __init__(self):
        self.board = [0] * 9 
        self.game_over = False
        self.winner = None

    def move(self, position, player):
        if self.board[position] == 0 and not self.game_over:
            self.board[position] = player
            self.check_game_over(player)

    def check_game_over(self, player):
        winning_positions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  
            [0, 4, 8], [2, 4, 6]             
        ]
        for positions in winning_positions:
            if all(self.board[pos] == player for pos in positions):
                self.game_over = True
                self.winner = player
                return
        if 0 not in self.board:
            self.game_over = True  # Draw

    def reset(self):
        self.board = [0] * 9
        self.game_over = False
        self.winner = None


In [4]:
def features(state):
    winning_combinations = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8], 
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  
        [0, 4, 8], [2, 4, 6]             
    ]
    x0 = 1
    x1 = x2 = x3 = x4 = x5 = x6 = 0

    x3 = 1 if state[4] == 1 else 0

    for corner in [0, 2, 6, 8]:
        if state[corner] == 1:
            x4 += 1

    for combo in winning_combinations:
        pieces = [state[i] for i in combo]
        if pieces.count(1) == 2 and pieces.count(0) == 1:
            x1 += 1
        elif pieces.count(-1) == 2 and pieces.count(0) == 1:
            x2 += 1
        elif pieces.count(1) == 1 and pieces.count(0) == 2:
            x5 += 1
        elif pieces.count(1) == 3:
            x6 += 1

    return [x0, x1, x2, x3, x4, x5, x6]

In [5]:
def value(state, weights):
    return sum(f*w for f, w in zip(features(state), weights))

In [6]:
def play_with_agent(game, weights):
    while not game.game_over:
        player_move = int(input("Your move (0-8): "))
        game.move(player_move, -1)
        if game.game_over:
            break
        best_move = None
        best_value = -float('inf')
        for move in range(9):
            if game.board[move] == 0:
                game_copy = deepcopy(game)
                game_copy.move(move, 1)
                move_value = value(game_copy.board, weights)
                if move_value > best_value:
                    best_value = move_value
                    best_move = move
        print(f"Agent moves at {best_move}")
        game.move(best_move, 1)

In [7]:
game = TicTacToe()
print("Starting a new game between two trained agents. Agent 1 is 'X', and Agent 2 is 'O'.")

def print_board(state):
    symbols = {0: " ", 1: "X", -1: "O"}
    print("\nBoard:")
    for i in range(3):
        print("|".join(symbols[state[j]] for j in range(i*3, i*3+3)))
        if i < 2:
            print("-----")

def agent_move(game, player, weights):
    best_move = None
    best_value = -float('inf') if player == 1 else float('inf')
    for move in range(9):
        if game.board[move] == 0:
            game_copy = deepcopy(game)
            game_copy.move(move, player)
            move_value = value(game_copy.board, weights)
            if (player == 1 and move_value > best_value) or (player == -1 and move_value < best_value):
                best_value = move_value
                best_move = move
    return best_move

def play_game(weights1, weights2):
    game = TicTacToe()
    player = 1 
    while not game.game_over:
        current_weights = weights1 if player == 1 else weights2
        move = agent_move(game, player, current_weights)
        game.move(move, player)
        print_board(game.board)
        if game.game_over:
            break
        player *= -1

    if game.winner is None:
        print("The game is a draw.")
    elif game.winner == 1:
        print("Agent 1 ('X') wins!")
    else:
        print("Agent 2 ('O') wins!")
number_of_games = 5
for i in range(number_of_games):
    print(f"Game {i + 1}:")
    if i % 2 == 0:
        play_game(weights_agent_1, weights_agent_2)
    else:
        play_game(weights_agent_2, weights_agent_1)

Starting a new game between two trained agents. Agent 1 is 'X', and Agent 2 is 'O'.
Game 1:

Board:
X| | 
-----
 | | 
-----
 | | 

Board:
X|O| 
-----
 | | 
-----
 | | 

Board:
X|O|X
-----
 | | 
-----
 | | 

Board:
X|O|X
-----
 |O| 
-----
 | | 

Board:
X|O|X
-----
 |O| 
-----
 |X| 

Board:
X|O|X
-----
O|O| 
-----
 |X| 

Board:
X|O|X
-----
O|O| 
-----
X|X| 

Board:
X|O|X
-----
O|O| 
-----
X|X|O

Board:
X|O|X
-----
O|O|X
-----
X|X|O
The game is a draw.
Game 2:

Board:
X| | 
-----
 | | 
-----
 | | 

Board:
X|O| 
-----
 | | 
-----
 | | 

Board:
X|O|X
-----
 | | 
-----
 | | 

Board:
X|O|X
-----
 |O| 
-----
 | | 

Board:
X|O|X
-----
 |O| 
-----
 |X| 

Board:
X|O|X
-----
O|O| 
-----
 |X| 

Board:
X|O|X
-----
O|O|X
-----
 |X| 

Board:
X|O|X
-----
O|O|X
-----
O|X| 

Board:
X|O|X
-----
O|O|X
-----
O|X|X
Agent 1 ('X') wins!
Game 3:

Board:
X| | 
-----
 | | 
-----
 | | 

Board:
X|O| 
-----
 | | 
-----
 | | 

Board:
X|O|X
-----
 | | 
-----
 | | 

Board:
X|O|X
-----
 |O| 
-----
 | | 

Board:
X|O|X
--